# Post Processing Analysis of 3D Subduction Simulations with Plotly

This script aim to use the output of the numerical Velocity field to extract the evolution of the vorticity field through time, for comparison with the analytical work on O'Connell 1992.

-------
RM- $\eta_M/ \eta_{WL} = 10^{0}$
![alt](testrm.gif) 

Case 1: this script wants to plot the value of the vorticity field in three different plane at the appropriate depth at a time step.
------
To characterize the induced flow dynamics we calculate the horizontal divergence and the vertical component of vorticity as:

\begin{equation*}
  \nabla_{h}=  \left( \frac{\partial}{\partial x},\frac{\partial}{\partial y} \right) \cdot \textbf{u}, \quad w_{z}=  \hat{z} \cdot \nabla \times \textbf{u}
\end{equation*}

In [ ]:
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np

import underworld as uw
import math
from underworld import function as fn

import os

import matplotlib.pyplot as pyplot

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

from scipy.signal import butter, lfilter, freqz, savgol_filter
from scipy.signal import wiener, medfilt
from matplotlib.colors import Normalize

import matplotlib.colors as colors
import matplotlib.cbook as cbook 

import plotly.graph_objects as go
import pandas as pd

In [ ]:
#==========================================================================
# Initial parameters for post-processing analysis
#==========================================================================
# Specify the directory in which you want to visualize the models
outputPath = os.path.join(os.path.abspath("."),"PostProcAnalyisis3drm/")

if uw.mpi.rank==0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
uw.mpi.barrier()

# Input parameters
printmod=1;             # Want the figures to be saved?
FiguresVisible = 'on';  # want to see the figures?

# Base directory
main_directory    = ('/home/jovyan/3D-Post_Processing/')

# Directory you wish to analise
ndirs             = [1]

# Get to the base directory
os.chdir(main_directory)
cur_dir           = os.getcwd()
print ('Current directory: ',cur_dir)
plt.figure().clf()

print ('-----------------------------')
print ('Start sampling the data:')

In [ ]:
#==========================================================================
# Simulation details
#==========================================================================
#==========================================================================
# Resolution
#==========================================================================
xRes      =  256
zRes      =  64
yRes      =  64
time         = 157.              # Initial time
#==========================================================================
# Characteristics Values          #
#==========================================================================
l_D            =   1.e5        #   [m] 100km
eta_D          =   1.e21       #   [Pa s]
sigma_D        =   1.e10       #   [Pa] 10 GPa ~ 250 km depth
Temperature_D  =   1000       #   [K]


#==========================================================================
# Reference choosen values
#==========================================================================
l_DL            =   1.
eta_DL          =   1.
sigma_DL        =   1.
Temperature_DL  =   1.


#==========================================================================
# Trivial Scaling Factor
#==========================================================================
l_SF            =   l_DL     / l_D
eta_SF          =   eta_DL   / eta_D
sigma_SF        =   sigma_DL / sigma_D
Temperature_SF  =   Temperature_DL / Temperature_D

#==========================================================================
# Basic scaling coefficient LAMEM
#==========================================================================
time_SF         =   eta_SF/sigma_SF
force_SF        =   sigma_SF*(l_SF*l_SF)
g_SF            =   l_SF/time_SF/time_SF
m_SF            =   force_SF/g_SF

#==========================================================================
# Other scaling coefficients
#==========================================================================
angle_SF           = math.pi/180;
area_SF            = l_SF*l_SF
volume_SF          = area_SF*l_SF
energy_SF          = force_SF*l_SF
power_SF           = energy_SF/time_SF
v_SF               = l_SF/time_SF
strain_rate_SF     = 1.0/time_SF
rho_SF             = m_SF/volume_SF

#==========================================================================
# Thermal scaling coefficients
#==========================================================================
Thermal_conductivity_SF         = power_SF/l_SF /Temperature_SF
Thermal_expansion_SF            = 1/Temperature_SF
capacity_SF                     = energy_SF/ Temperature_SF / m_SF
Thermal_diffusivity_SF          = (l_SF*l_SF)  /  time_SF #in uw2

#==========================================================================
# Additional UNITS
#==========================================================================
yr     = 3600.0*24.0*365.25;
Myr    = 1e6*yr;

km     = 1e3;
cm     = 1.e-2;
cm_yr  = cm/yr;
MPa    = 1e6;
mW     = 1e-3;
K      =  273.15
#==========================================================================
# Dimensional physical data
#==========================================================================
DII_ref        =   1e-16
boxWidth       =    3000.0*km
boxHeight      =    660.0*km
boxLength      =    4000.0*km
boxHeight      =    660.0*km
l_max_op       =    1750.0*km
l_min_op       =    - 1000.0*km

Depth_slab     =    - 256.2536*km
l_op           =    - 310.0*km

#==========================================================================
# Getting dimension-less values
#==========================================================================

boxWidth_DL      =  boxWidth * l_SF
boxLength_DL     =  boxLength * l_SF
boxHeight_DL     =  boxHeight * l_SF
l_max_op_DL      =  l_max_op * l_SF
l_min_op_DL      =  l_min_op * l_SF
Depth_slab_DL    =  Depth_slab * l_SF
l_op_DL          =  l_op * l_SF

dim       =  3

L         =  boxLength_DL
W         =  boxWidth_DL
H         =  boxHeight_DL

dx         =   boxLength_DL/xRes
dz         =   boxHeight_DL/zRes
dy         =   W/yRes
dy         =     W/yRes

y_back     =    -W/2;                #  coord of the front margin
w_min_op   =    -W               # attached to the front wall
w_max_wl   =    -W/2 + W/2;
w_max_slab =    -y_back
x_left     =   -L/2                 #  coord of the left margin
z_bot      =   -H                   #  coord of the bottom margin
z_surface  =   -H+H
#modelTime = str("{0:.5f}").format(float(time)/ time_SF/ Myr/10000 )

gridpoints_x = xRes+1
gridpoints_z = zRes+1
gridpoints_y = yRes+1
gridpoints_xy=gridpoints_x*gridpoints_y
gridpoints_xz=gridpoints_x*gridpoints_z

In [ ]:
#==========================================================================
# INITIALIZE MESH AND MESH VARIABLES
#==========================================================================
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"),
                                elementRes  = (xRes, yRes, zRes),
                                minCoord    = (x_left , z_bot, y_back),
                                maxCoord    = (-x_left, z_surface, -y_back) )

velocityField       = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=dim )
pressureField       = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )
temperatureField    = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=1 )

In [ ]:
c = 0
time_sol_perdir = []
steps_per_dir = []

# 1st loop over the different simulations directories
for i in ndirs:   
    print (i)
    sim_directory  = ('m'+ str(ndirs[c]))    
    sim_path       = main_directory +sim_directory      

    output_path  = sim_path
    os.chdir(output_path)
       
    # 1st if outputdir exisit
    if  output_path: 
        
        f = h5.File("vel-0.00048.h5", "r")
        if f:
            print ("found it")
        # Get and print list of datasets within the H5 file
            datasetNames = [n for n in f.keys()]
            for n in datasetNames:
                print(n)

In [ ]:
#==========================================================================
# get some scaling done
#==========================================================================
def lenkm(l):
    myarray = np.array(l)
    lengthDim = []
    lengthDim = [np.array(i) /l_SF /km  for i in l]
    return lengthDim

def timeMyr(t):
    myarray = np.array(t)
    timeDim = []
    timeDim[:] = [ np.array(i) / time_SF/ Myr for i in myarray]   
    return timeDim

def VelcmYr(v):
    myarray = np.array(v)
    VelDim = []
    VelDim = [np.array(i) /v_SF* (yr/cm) for i in myarray]
    return VelDim

def strain_fn(minus_s):
    myarray = np.array(minus_s)
    VelDim = []
    VelDim = [np.array(i) /strain_rate_SF for i in myarray]
    return VelDim

Berc_SF = 1e9*yr;

def Bercov(values):
    myarray = np.array(values)
    BercApprox= []
    BercApprox = [np.array(i) *(time_SF* Berc_SF) for i in myarray]
    return BercApprox

def returnarrayoffloats(DL_list):
    myarray = np.array(DL_list)
    DL_array = []
    DL_array = [np.array(i) for i in myarray]
    return DL_array

In [ ]:
# load the velocity field from the h5 file
print (mesh.elementRes)
print (f.filename)
vel = mesh.add_variable(3)
vel.load(str(f.filename))

# the underworld (uw) function from (Moresi, 2003) allow us to extract gradients and interpolatie functions at the mesh nodes in an MPI friendly env

In [ ]:
# projects the gradient of the velocity field on the mesh nodes to decrease numerical noise
projected_fn_gradient = mesh.add_variable(9)
gradient_projector = uw.utils.MeshVariable_Projection(projected_fn_gradient,vel.fn_gradient,type=0)
gradient_projector.solve()

In [ ]:
#extract vorticty and divergence from the gradient function -do note that the cartesian plane is oriented as x-z-y
w_z                 =   (projected_fn_gradient[6] - projected_fn_gradient[2])
div_h               =  (projected_fn_gradient[0] - projected_fn_gradient[8]) 

# The function w_z is here evaluated over the full mesh.

In [ ]:
# evaluate vorticity at different surface plans and dimensionalised in Gyr-1
evaluated_w =  w_z.evaluate_global(mesh.data)
w_z_glob = np.array(Bercov(evaluated_w ))

In [ ]:
# Create 3D grid
X      =        np.linspace(-2000., 2000.,xRes+1)
Y      =        np.linspace(-1500., 1500., zRes+1)
Z      =        np.linspace(-660., 0., yRes+1)
x, y, z = np.meshgrid(X, Y, Z, indexing='ij')

w_z_glob_reshaped = np.reshape(w_z_glob,(257,65,65))  

#Alternatively these are the coords extracted from the mesh itself in the same format and right SI
xcoords = mesh.data[:,0]*l_SF/km
zcorrds = mesh.data[:,1]*l_SF/km
ycoords = mesh.data[:,2]/l_SF/km

In [ ]:
Function = np.arange(1085825)  #generating example data
Function_shaped = np.reshape(Function,(257,65,65))   

mycolorscale = [[0, 'magenta'],[0.3, 'violet'], [0.35, 'blue'], [0.5, 'green'],[0.6, 'yellow'],[0.8, 'orange'], [1, 'red']]


fig1 = go.Figure(data=go.Isosurface(   
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=Function_shaped.flatten(),
    #isomin=0,
    #isomax=50,
    #surface_fill=0.4,
    opacity=0.6,
    coloraxis='coloraxis',
    caps=dict(x_show=False, y_show=False),
    slices_z=dict(show=True, locations=[0, -110]),
    #slices_y=dict(show=True, locations=[-2000]),
    ))

# define your plot ennv
fig1.update_layout(coloraxis = dict(colorscale=mycolorscale, cmin =-320, cmax=320, colorbar_thickness=25))

fig1.update_layout(
    margin=dict(t=0, l=0, b=0), # tight layout
    scene_camera_eye=dict(x=1.86, y=0.61, z=0.98))
#fig1.show()

In [ ]:
xcoords_reshaped  = np.reshape(xcoords,(257,65,65))  
zcorrds_reshaped  = np.reshape(zcorrds,(257,65,65))  
ycoords_reshaped  = np.reshape(ycoords,(257,65,65))  

#Proper problem

In [ ]:
fig1 = go.Figure(data=go.Isosurface(   
    x=xcoords_reshaped.flatten(),
    y=ycoords_reshaped .flatten(),
    z=zcorrds_reshaped.flatten(),
    value=w_z_glob_reshaped .flatten(),
    #isomin=0,
    #isomax=50,
    #surface_fill=0.4,
    opacity=0.6,
    coloraxis='coloraxis',
    caps=dict(x_show=False, y_show=False),
    slices_z=dict(show=True, locations=[0, -110]),
    #slices_y=dict(show=True, locations=[-2000]),
    ))

# define your plot ennv
fig1.update_layout(coloraxis = dict(colorscale=mycolorscale, cmin =-320, cmax=320, colorbar_thickness=25))

fig1.update_layout(
    margin=dict(t=0, l=0, b=0), # tight layout
    scene_camera_eye=dict(x=1.86, y=0.61, z=0.98))
#fig1.show()
